In [1]:
import sys
import os
sys.path.append('../')

In [2]:
import torch
from torch import nn
import numpy as np
import dataset
import importlib

In [3]:
import dataset
importlib.reload(dataset)

<module 'dataset' from 'g:\\Dev\\AI\\Projects\\ImageGeneration\\dataset.py'>

In [ ]:
dataset = dataset